**классификатор текстов LSTM на Keras+TensorFlow**

Евгений Борисов <borisov.e@solarl.ru>

https://habr.com/ru/company/dca/blog/274027/    
http://help.sentiment140.com/for-students/   
http://study.mokoron.com  

## Библиотеки

In [14]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 200  
import re
# import gzip
from tqdm import tqdm

In [15]:
tqdm.pandas()

/usr/lib/python3.9/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

## Данные

In [3]:
ff = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount','unk']

In [4]:
neg = pd.read_csv('../data/twit/negative.csv.gz',sep=';',header=None)
ppr(neg)
neg.columns = ff

записей: 111 923


In [5]:
pos = pd.read_csv('../data/twit/positive.csv.gz',sep=';')
ppr(pos)
pos.columns = ff

записей: 114 910


In [6]:
data = pd.concat([pos,neg],sort=False)[['id','ttext', 'ttype']]
ppr(data)

записей: 226 833


In [7]:
data.sample(10)

,id,ttext,ttype
53060,410062212502667264,"Да, не спорю ;) грозного взгляда инструктора же не будет ""RT @arzik2: @M_lleBarankina :)) http://t.co/M4Q6qbIeY1""",1
97589,422810176802942976,"омг, яникс в Минске, лучше бы Билли приехал(",-1
22537,409434898890358784,@fixmelater так ты с нами из-за уборки не идешь? давай. нам нужен четвертый человек),1
57607,416097158811885568,"RT @kalinina1907: ну к чему опять эти сны,а?:(",-1
78749,410724034448338945,"Думаю, каникулы чинуш в Куршевелях уже испорчены: невозможно же будет даже расслабиться - а вдруг за тобой наблюдает журналист Навального? )",1
91850,422020780197564416,"@katya_colfer очень. в школе уроки сложные были, а потом пошли в студию фоткаться на альбоом((",-1
73109,418307802780532736,RT @russian_loser: @PastorAfuckingA Это кто еще БОЯТЬ чью жизнь пиздит :|,-1
104247,423970538617581568,"RT @aracetiruo: Кстати, в отпуске прочитал, как описал вампирский сериальчик. Даже посмотреть хочется, а то от BSG рвотные позывы :(",-1
79253,419440884590313472,"37.5,потом 38.5,пойдёт выше-скорую вызывать будем:|",-1
31145,409651639407378432,RT @elkaundead: а снега то ещё больше мимими)))\nМатвей держись),1


## токенизация и очистка

In [8]:
from pymorphy2 import MorphAnalyzer

In [9]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

In [10]:
# лемматизация и очистка с помощью пакета морфологического анализа

morph = MorphAnalyzer()

# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

# NOUN (существительное), VERB (глагол), ADJF (прилагательное)
def word_normalizer(w, pos_types=('NOUN','VERB','ADJF')):
    if not morph.word_is_known(w): return ''
    p = morph.parse(w)[0] 
    return p.normal_form if (p.tag.POS in pos_types) else ''


def tokenize_normalize(s):
    return [ word_normalizer(w) for w in s.split(' ') if len(w)>1 ]

In [ ]:
data['ctext'] = data['ttext'].progress_apply(string_normalizer).progress_apply( tokenize_normalize )

 11%|█         | 24495/226833 [00:24<03:21, 1002.64it/s]

In [ ]:
vcb0 =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb0)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

In [ ]:
data['ctext'] = data['ctext'].apply( ' '.join  )

In [ ]:
data.sample(10)

## очистка данных

In [ ]:
data['ttext_clean'] = data['ttext'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'^http.*',' url ', w.strip() ) for w in t  ]
  )

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]-*[)D]',' happysmile ', w.strip() )for w in t ]
  )

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\)\)\)*',' happysmile ', w.strip() ) for w in t ]
  )

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]\*',' kisssmile ', w.strip() ) for w in t ]
  )

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r':\(',' sadsmile ', w.strip() ) for w in t ]
  )

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\(\(\(*',' sadsmile ', w.strip() ) for w in t ]
  )

In [ ]:
data['ttext_clean'] = [ ' '.join(s) for s in data['ttext_clean'] ]

In [ ]:
data['ttext_clean'] = data['ttext_clean'].str.lower()
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'_', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 


In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [ ]:
# замена буквенно-цифровых кодов
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: [w for w in t if not re.match( r'\b.*\d+.*\b', w) ]
)

In [ ]:
# data[['ttext_clean']]
# data[['ttext']]

---

In [ ]:
# from nltk import download as nltk_download
# nltk_download('stopwords')

# from Stemmer import Stemmer

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords

stopwords = set(nltk_stopwords.words('russian') )

In [ ]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

ppr(stopwords)

In [ ]:
# удаление лишних слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if w not in stopwords])

In [ ]:
%xdel stopwords

In [ ]:
# %%time 

# from Stemmer import Stemmer
# # pacman -S python-pystemmer
# # pip install pystemmer

# # стемминг, выделение основы слова
# data['ttext_clean'] = data['ttext_clean'].apply( lambda t:Stemmer('russian').stemWords(t) )

In [ ]:
# удаление коротких слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if len(w)>2])

---

In [ ]:
# data[ data['ttext_clean'].str.len()<1 ][['ttext_clean']]

In [ ]:
ppr(data)
data = data[ data['ttext_clean'].str.len()>0 ].reset_index(drop=True) 
ppr(data)

In [ ]:
data.sample(3)

## строим датасет

In [ ]:
vocab = ['<PAD>','<START>','<UNK>'] + sorted(set([ w for t in data['ttext_clean'] for w in t if w ]))
ppr(vocab)

In [ ]:
# %%time

# from gensim.models.word2vec import Word2Vec

# w2v = Word2Vec( common_texts, min_count=1, size=256, window=4, workers=4)

# # with open('result/Word2Vec.pkl', 'wb') as f: pickle.dump(w2v, f)

In [ ]:
vocab = { w:n for n,w in enumerate(vocab) }

---

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply( lambda d: d+['<START>'] )

In [ ]:
n_max = data['ttext_clean'].str.len().max()
n_max

In [ ]:
pad = ['<PAD>']*n_max

In [ ]:
data[['ttext_clean']]

In [ ]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: pad[len(t):] + list(reversed(t)) 
  )

In [ ]:
data[['ttext_clean']]

In [ ]:
data['ttext_code'] = data['ttext_clean'].apply(lambda t: [ vocab[w] for w in t ] )

In [ ]:
data['ttext_code'].values

In [ ]:
len(data)//32

In [ ]:
ppr(data)
data = data.sample(32*7088).reset_index(drop=True)
ppr(data)


---

In [ ]:
X = np.stack( data['ttext_code'].values).astype(np.float32 ) # , axis=-1)
X.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder

y = data['ttype'].values
y = OneHotEncoder(categories='auto').fit_transform(y.reshape(-1,1) ).todense().astype(np.float32)
y.shape


In [ ]:
# np.save('X.npy',X)
# np.save('y.npy',y)

In [ ]:
# import numpy as np

# X = np.load('X.npy')
# y = np.load('y.npy')

In [ ]:
vocab_size = int(X.max())
X.shape , y.shape, vocab_size

## строим нейросеть 

In [ ]:
# import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
# n=226826
# for i in range(1,n//2):
#     if n%i==0: print(i)
# # 23
# # 46
# # 4931
# # 9862

In [ ]:
time_steps=X.shape[1]
batch_size=32
num_classes=y.shape[1]

vocab_size = len(vocab)

In [ ]:
embedding_size=64

model = Sequential()

model.add(Embedding(
       input_dim=vocab_size, # e.g, 10 if you have 10 words in your vocabulary
       output_dim=embedding_size, # size of the embedded vectors
       input_length=time_steps,
       batch_input_shape=(batch_size,time_steps)
    ))

model.add(LSTM(
       32, 
       return_sequences=False, 
       stateful=False)
    )

model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
%%time

model.fit(X,y, batch_size=batch_size, epochs=1, )

In [ ]:
# score = model.evaluate(X,y)